In this Notebook we will demontrate text classification model using Glove embeddings with Neural network.

In [1]:
import pandas as pd 
import numpy as np 
import sys 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential
from keras.initializers import Constant 
import zipfile
import os 
import nltk
import tensorflow as tf
from bs4 import BeautifulSoup
import regex as re 
from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
! wget https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Gradient-Boosting-with-XGBoost-and-Scikit-learn/master/Chapter02/heart_disease.csv

In [ ]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
with zipfile.ZipFile('/content/glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Glove')

In [2]:
train_df = pd.read_csv('/content/IMDB Dataset.csv')

In [3]:
train_df.shape[0]

50000

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

def clean_text(text):
  text = str(text)
  for punc in puncts:
      if punc in text:
          text = text.replace(punc, ' ')
  return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column

train_df['review'] = train_df['review'].apply(lambda x: remove_emoji(x))
train_df['review'] = train_df['review'].apply(lambda x: clean_text(x))
train_df['review'] = train_df['review'].apply(lambda x: denoise_text(x))
train_df['review'] = train_df['review'].apply(lambda x: remove_special_characters(x))
train_df['review'] = train_df['review'].apply(lambda x: simple_stemmer(x))
train_df['review'] = train_df['review'] .apply(lambda x: x.lower())

In [ ]:
# #Preprocessing the test dataset as well
# test_df['tweet'] = test_df['tweet'].apply(lambda x: remove_emoji(x)) 
# test_df['tweet'] = test_df['tweet'].apply(lambda x: clean_text(x)) 
# test_df['tweet'] = test_df['tweet'].apply(lambda x: re.sub(r'http\S+','',x))
# test_df['tweet'] = test_df['tweet'].apply(lambda x: re.sub("@[\w]*", '', x))
# test_df['tweet'] = test_df['tweet'].apply(lambda x:' '.join(x.split()))
# test_df['tweet'] = test_df['tweet'].apply(lambda x: remove_stopwords(x))
# test_df['tweet'] = test_df['tweet'].apply(lambda x: x.lower())


In [ ]:
train_df.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl product br br the film techniqu...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there s a famili where a littl boy jake ...,negative
4,petter mattei s love in the time of money is a...,positive


In [ ]:
GLOVE_DIR = '/content/Glove'

MAX_LENGTH_SEQ = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.20
LABELS_LEN = train_df['sentiment'].nunique()

In [ ]:
train_df['sentiment'] = train_df['sentiment'].map({'positive':1,'negative':0})

In [ ]:
X = train_df['review']
y = train_df['sentiment']

## Loading and Preprocessing 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=2018)

In [ ]:
##Vectorize these text samples into a 2D integer tensor using Keras Tokenizer
#Tokenizer is fit on training data only, and that is used to tokenize both train and test data.
tokenizer = Tokenizer(num_words = MAX_NUM_WORDS )
tokenizer.fit_on_texts(train_df['review'])
train_sequences =   tokenizer.texts_to_sequences(train_df['review'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 71419 unique tokens.


In [ ]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
#initial padding of 0s, until vector is of size MAX_SEQUENCE_LENGTH
train_data = pad_sequences(train_sequences, maxlen=MAX_LENGTH_SEQ)
train_labels = to_categorical(np.asarray(train_df['sentiment']))

In [ ]:
#split the training data into a training se and validation set 
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * train_data.shape[0])
x_train = train_data[:-num_validation_samples]
y_train = train_labels[:-num_validation_samples]
x_val = train_data[-num_validation_samples:]
y_val = train_labels[-num_validation_samples:]

In [ ]:
print('Preparing embedding matrix')

#First build index mapping words in the embedding set
#to their embedding vector 

embedding_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embedding_index))
print(embedding_index["google"])

Preparing embedding matrix
Found 400000 word vectors in Glove embeddings.
[ 0.22575  -0.56253  -0.05156  -0.079389  1.1876   -0.48397  -0.23342
 -0.85278   0.97495  -0.33344   0.71692   0.12644   0.31962  -1.4136
 -0.57903  -0.037286 -0.0164    0.45155  -0.29005   0.52599  -0.22534
 -0.29556  -0.032407  1.5608   -0.013499 -0.064558  0.26625   0.78595
 -0.71693  -0.93025   0.80461   1.6035   -0.30602  -0.34764   0.93872
  0.38137  -0.26743  -0.56519   0.58899  -0.14554  -0.34324   0.21291
 -0.39887   0.090042 -0.8495    0.38803  -0.5045   -0.22488   1.0644
 -0.2624    1.0334    0.06348  -0.39989   0.24236  -0.65636  -1.8107
 -0.061801  0.13795   1.1658   -0.30046  -0.50143   0.16509   0.039835
  0.62541   0.56935   0.64125   0.21308   0.30276   0.39673   0.38973
  0.28183   0.79481  -0.11962  -0.49598  -0.53195  -0.14897   0.51254
 -0.39208  -0.58535  -0.078509  0.81721  -0.73497  -0.68131   0.099243
 -0.87608   0.029632  0.33402  -0.14305   0.16964  -0.035178  0.39777
  0.71769   0.258

In [ ]:
#Prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from the Glove
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
  print(i, word)
  if i > MAX_NUM_WORDS:
    continue
  embedding_vector = embedding_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length=MAX_LENGTH_SEQ,
                            trainable=False)
print('Preparing of embedding matrix is done')

Preparing of embedding matrix is done



**1D CNN Model with pre-trained embedding¶** 

In [ ]:
cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(LABELS_LEN, activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
cnnmodel.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_val,y_val))

**LSTM Model with training your own embedding**

In [ ]:
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(2, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))

Training the RNN
198/198 [==============================] - 290s 1s/step - loss: 0.4286 - accuracy: 0.7985 - val_loss: 0.2342 - val_accuracy: 0.9028


**Bidirectional LSTM with 3 Output Layers**

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Bidirectional(LSTM(128, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(LSTM(128, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu', kernel_initializer='uniform'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data= (x_val, y_val))

Epoch 1/5
1250/1250 [==============================] - 341s 243ms/step - loss: 0.6434 - accuracy: 0.6196 - val_loss: 0.4753 - val_accuracy: 0.7807
Epoch 2/5
1250/1250 [==============================] - 307s 245ms/step - loss: 0.4436 - accuracy: 0.7985 - val_loss: 0.3591 - val_accuracy: 0.8463
Epoch 3/5
1250/1250 [==============================] - 308s 246ms/step - loss: 0.3555 - accuracy: 0.8458 - val_loss: 0.3241 - val_accuracy: 0.8578
Epoch 4/5
1250/1250 [==============================] - 308s 246ms/step - loss: 0.3066 - accuracy: 0.8651 - val_loss: 0.3039 - val_accuracy: 0.8712
Epoch 5/5
1250/1250 [==============================] - 308s 246ms/step - loss: 0.2812 - accuracy: 0.8783 - val_loss: 0.3238 - val_accuracy: 0.8569


In [ ]:
test = ['it was a very bad movie']
test = tokenizer.texts_to_sequences(test)
test = pad_sequences(test, maxlen=train_df['review'].shape[0], dtype='int32', value=0)

In [ ]:
sentiment = model.predict(test)[0]
if (np.argmax(sentiment) == 0):
  print('Negative')
elif (np.argmax(sentiment) == 1):
  print('positive') 

Negative
